In [4]:
import requests, graphviz, textwrap
from lxml import html

nodelayout = ""

In [5]:
g = graphviz.Graph()

In [73]:
#response = requests.get("http://www.clickhole.com/clickventure/would-you-survive-bear-attack-1230")
#response = requests.get("http://www.clickhole.com/clickventure/can-you-survive-seeing-grease-broadway-4331#2,")
response = requests.get("http://www.clickhole.com/clickventure/mysterious-shadows-skullshadow-island-1880#2,")

In [74]:
tree = html.fromstring(response.content)

In [80]:
data_dict = dict()
for a in tree.xpath('//div[contains(@class, "clickventure-node ")]'):
    #print list(a.classes)
    text = a.xpath('.//p')
    #print dir(a.xpath('.//img')[0]), a.xpath('.//img')[0].items()[0][1]
    #print [t.text for t in text]
    allparagraphtext = '\n'.join(t.text.encode('utf-8', 'ignore') for t in text if t.text is not None)
    #print allparagraphtext
    thisid = a.xpath('.//@data-node-id')[0]
    thisdict = {'text': allparagraphtext, 'links' : list()}
    if 'clickventure-node-start' in list(a.classes):
        thisdict['type'] = 'start'
    elif 'clickventure-node-finish' in list(a.classes):
        thisdict['type'] = 'finish'
    else:
        thisdict['type'] = 'normal'
        
    if len(a.xpath('.//img')):
        thisdict['img'] = a.xpath('.//img')[0].items()[0][1]
    else:
        thisdict['img'] = None
    data_dict[int(thisid)] = thisdict
    #print int(thisid[0])
    #print text.text
    for b in a.xpath('.//div[contains(@class, "clickventure-node-link ")]'):
        linkid =  int(b.xpath('.//@data-target-node')[0])
        linktext = b.text_content().strip()
        data_dict[int(thisid)]['links'].append((int(linkid), linktext.encode('ascii', 'ignore')))
        #print b.text_content().strip() + " --> " + str(linkid)
    #print

In [81]:
g = graphviz.Graph()
for k,v in data_dict.iteritems():
    #print type(v['text'])
    wrappedtext = '\n'.join(textwrap.wrap(v['text'],60))
    g.node(str(k), label="<p>hi</p></br>" + wrappedtext.decode('utf-8'))
    #print v
for k,v in data_dict.iteritems():
    for link in v['links']:
        g.edge(str(k), str(link[0]), link[1])